In [1]:
import sys
sys.dont_write_bytecode = True
from collections import OrderedDict
import multiprocessing
import math
from breeder_blanket_model_maker import *
from breeder_blanket_model_maker.rewrite_cad_files import *
from breeder_blanket_model_maker.serpent_tools import *
from breeder_blanket_model_maker.make_serpent_input_file import *

from neutronics_material_maker.nmm import *
from neutronics_material_maker.examples import *


example materials imported
<< SS316LN-IG >> 
<< Made with https://github.com/ukaea/neutronics_material_maker >> 
<< density =0.0858294 atoms per barn cm2 >> 
<< temperature =293.15 K >> 
DENSITY 7.93
FUEL 46
Fe54 1.273249513212e+22
Fe56 1.998729441150e+23
Fe57 4.615937927281e+21
Fe58 6.142966000441e+20
C12 4.720236236010e+20
C13 5.105279260620e+18
Mn55 6.954099361314e+21
Si28 3.136253783687e+21
Si29 1.593241271328e+20
Si30 1.051505231792e+20
P31 1.541807450070e+20
S32 5.658920738413e+19
S33 4.468039323939e+17
S34 2.531888950232e+18
S36 5.957385765252e+15
Cr50 2.873262722994e+21
Cr52 5.540801157581e+22
Cr53 6.282823735595e+21
Cr54 1.563927811250e+21
Ni58 2.769527191816e+22
Ni60 1.066811280623e+22
Ni61 4.637372454648e+20
Ni62 1.478637944001e+21
Ni64 3.765144492304e+20
Mo92 7.809517174200e+20
Mo94 4.917899665790e+20
Mo95 8.513609913236e+20
Mo96 8.959714473084e+20
Mo97 5.159763583779e+20
Mo98 1.310902435504e+21
Mo100 5.278008165908e+20
N14 1.087062295923e+21
N15 3.971362516721e+18
B10 3.51

In [3]:
def define_blanket_geometry_parmeters(blanket_type,input_files,output_directory,poloidal_lithium_lead_in_mm = 35.4):

    poloidal_lithium_lead_in_mm = float(poloidal_lithium_lead_in_mm)
    smd = 274. * 1000000.  # units Pa

    cooling_plates_channel_poloidal_mm = 3.0
    cooling_plates_channel_radial_mm = 3.0
    cooling_plates_channel_toroidal_mm = 6.0
    first_wall_channel_radial_mm = 10.0
    first_wall_channel_poloidal_mm = 15.0
    first_wall_channel_pitch_mm = 5.2

    pressure_coolant = 10. * 1000000.  # in units Pa

    first_walls_thickness = (first_wall_channel_radial_mm/1000) + math.sqrt((pressure_coolant * (poloidal_lithium_lead_in_mm/1000) * (poloidal_lithium_lead_in_mm/1000) ) / (4. * smd))  # units in m
    first_walls_thickness_mm = first_walls_thickness *1000

    poloidal_cooling_plate_mm = (pressure_coolant * (poloidal_lithium_lead_in_mm)) / (1.1 * smd)

    poloidal_cooling_plate_mm = max(poloidal_cooling_plate_mm, 0.002*1000) + cooling_plates_channel_poloidal_mm

    cooling_channel_offset_from_first_wall = (first_walls_thickness_mm - first_wall_channel_radial_mm) / 2.0

    if cooling_channel_offset_from_first_wall < 1.0:
        raise ValueError('first wall is too thin at ',cooling_channel_offset_from_first_wall, 'mm')

    poloidal_segmentations_ordered_dict=OrderedDict()
    poloidal_segmentations_ordered_dict['lithium_lead']=poloidal_lithium_lead_in_mm
    poloidal_segmentations_ordered_dict['cooling_plate_homogenised']=poloidal_cooling_plate_mm

    back_plates_thicknesses_ordered_dict=OrderedDict()  #these are special ordereddict types required in python 2.7 but in python 3.6 onwards dictionaries are ordered by default, sadly freecad is not yet avaialbe in python 3
    back_plates_thicknesses_ordered_dict['back_plate_1']=30
    back_plates_thicknesses_ordered_dict['back_helium']=50
    back_plates_thicknesses_ordered_dict['back_plate_2']=30
    back_plates_thicknesses_ordered_dict['back_lithium_lead']=25
    back_plates_thicknesses_ordered_dict['back_plate_3']=10

    first_wall_channel_poloidal_segmentations_dict=OrderedDict()
    first_wall_channel_poloidal_segmentations_dict['first_wall_material']=first_wall_channel_poloidal_mm
    first_wall_channel_poloidal_segmentations_dict['first_wall_coolant']=first_wall_channel_pitch_mm


    list_of_compressed_arguments=[]
    for module in input_files:

        blanket_geometry_parameters =  {

            'blanket_type' : blanket_type,
            'plasma_filename' :'/home/jshim/Eurofusion_baseline_2016/envelopes/plasma.step',
            'envelope_filename' : module,
            'output_folder' : output_directory,
            'output_files':['step','stl'],
            'first_wall_poloidal_fillet_radius' : 50,
            'armour_thickness' : 2,
            'first_wall_thickness' : first_walls_thickness_mm,
            'end_cap_thickness' : 25,
            'back_plates_thicknesses' : back_plates_thicknesses_ordered_dict,
            'poloidal_segmentations' : poloidal_segmentations_ordered_dict,

        }

        if module == input_files[13]:
            blanket_geometry_parameters.update({
                #The following three arguments are optional, if included the model will include cooling channels on the first wall
                'cooling_channel_offset_from_first_wall': cooling_channel_offset_from_first_wall,
                'first_wall_channel_radial_mm': first_wall_channel_radial_mm,
                'first_wall_channel_poloidal_segmentations': first_wall_channel_poloidal_segmentations_dict,  # 13.5,4.5

                # #The following two arguments are optional, if included the model will include a slice of blanket and cooling channels in the cooling plate
                'cooling_plates_channel_poloidal_mm':cooling_plates_channel_poloidal_mm,
                'cooling_plates_channel_radial_mm':cooling_plates_channel_radial_mm,

                'output_files':['step','stl','umesh'],
                })



        list_of_compressed_arguments.append(blanket_geometry_parameters)

    return list_of_compressed_arguments

def define_neutronics_materials(enrichment_fraction):

    mat_lithium_lead =Compound('Pb84.2Li15.8',
                              density_atoms_per_barn_per_cm=3.2720171E-2,
                              enriched_isotopes=[Isotope('Li',6,abundance=enrichment_fraction),
                                                 Isotope('Li',7,abundance=1.0-enrichment_fraction)])

    material_dictionary = {  'armour': mat_Tungsten,
                             'lithium_lead': mat_lithium_lead,
                             'back_plate_1': mat_Eurofer,
                             'back_plate_2': mat_Eurofer,
                             'back_plate_3': mat_Eurofer,
                             'back_lithium_lead': mat_lithium_lead,
                             'back_helium': mat_He_coolant_back_plate,
                             'cooling_plate_homogenised': mat_cooling_plates_homogenised,
                             'end_caps_homogenised': mat_end_caps_homogenised,
                             'first_wall_homogenised': mat_first_wall_homogenised,
                             'plasma': mat_DT_plasma,
                             'central_solenoid': mat_central_solenoid_m25,
                             'divertor_1st_layer': mat_divertor_layer_1_m15, 
                             'divertor_2nd_layer': mat_divertor_layer_2_m74, 
                             'divertor_3rd_layer': mat_divertor_layer_3_m15,
                             'divertor_4th_layer': mat_divertor_layer_4_m75,
                             'manifolder': mat_VV_Body_m60 ,
                             'ports': mat_TF_Casing_m50,
                             #'shell': , # outer shell ignored
                             'shield': mat_TF_Casing_m50,
                             'tf_case': mat_TF_Casing_m50,
                             'tf_coils': mat_TF_Magnet_m25,
                             'vacuum_1st_layer': mat_VV_Shell_m50,
                             'vacuum_2nd_layer': mat_VV_Body_m60,
                             'vacuum_3rd_layer': mat_VV_Shell_m50,
                             'vaccum_vessel_shield': mat_ShieldPort_m60,
                             'blanket_support':mat_Eurofer,

                             'first_wall_coolant':mat_He_in_first_walls,
                             'first_wall_material':mat_He_in_first_walls,

                             'slice_lithium_lead':mat_lithium_lead,
                             'slice_armour':mat_Tungsten,
                             'slice_first_wall_material':mat_Eurofer,
                             'slice_first_wall_homogenised':mat_first_wall_homogenised, #ignore
                             'slice_first_wall_coolant':mat_He_in_first_walls,
                             'slice_cooling_plate_material':mat_cooling_plates_homogenised,
                             'slice_cooling_plate_coolant':mat_He_in_coolant_plates,
                             'slice_back_plate_1':mat_Eurofer,
                             'slice_back_helium':mat_He_coolant_back_plate,
                             'slice_back_plate_2':mat_Eurofer,
                             'slice_back_lithium_lead':mat_lithium_lead,
                             'slice_back_plate_3':mat_Eurofer,
                          }
    return material_dictionary

def define_neutronics_model_parmeters(list_detailed_modules_parts,material_dictionary,output_directory,nps=1e7,**kwargs):

    neutronics_parameters= { 'output_folder':output_directory,
                             'parts':list_detailed_modules_parts,
                             'include_umesh':True,
                             'output_folder_stl':os.path.join(output_directory,'stl'),
                             'material_dictionary':material_dictionary,
                             'plot_serpent_geometry':True,
                             'nps':nps,
                             'tallies':[{'name': 'tbr',
                                         'bodies': ['lithium_lead'],
                                         'mt_number':-55,
                                         'particle_type':'n'},
                                        {'name': 'neutron_heating',
                                         'bodies': ['armour', 'lithium_lead', 'back_plate_1','back_plate_2','back_plate_3','back_lithium_lead','back_helium','cooling_plate_homogenised','end_caps_homogenised','first_wall_homogenised',],
                                         'mt_number':-4,
                                         'particle_type':'n'},
                                        {'name': 'photon_heating',
                                         'bodies': ['armour', 'lithium_lead', 'back_plate_1','back_plate_2','back_plate_3','back_lithium_lead','back_helium','cooling_plate_homogenised','end_caps_homogenised','first_wall_homogenised',],
                                         'mt_number':-26,
                                         'particle_type':'p'},
                                       ]
                             }

    return neutronics_parameters

In [5]:

output_directory='/home/jshim/detailed_HCLL'
list_of_geometry_parameters = define_blanket_geometry_parmeters(blanket_type ='HCLL',
                                                                input_files= ['/home/jshim/Eurofusion_baseline_2016/envelopes/mod' + str(x) + '.step' for x in range(1,27)],#27,
                                                                output_directory = output_directory,
                                                                poloidal_lithium_lead_in_mm =80.0)
list_of_detailed_modules_parts = detailed_module(list_of_geometry_parameters)


Creating detailed HCLL
'read_in_envelope_file'  5.34 ms
'read_in_envelope_file'  12.21 ms
'read_in_envelope_file'  5.56 ms
'read_in_envelope_file'  9.75 ms
'read_in_envelope_file'  9.91 ms
'read_in_envelope_file'  9.87 ms
'read_in_envelope_file'  10.46 ms
'find_envelope_back_face'  582.99 ms
'find_envelope_front_face'  9.61 ms
'find_front_face_midpoint'  0.22 ms
'envelope_front_face_id'  9.61 ms
'find_envelope_back_face'  832.53 ms
'find_envelope_front_face'  9.91 ms
'find_front_face_midpoint'  0.26 ms
'envelope_front_face_id'  11.62 ms
'find_envelope_back_face_id'  480.70 ms
'find_front_face_polodial_edges_to_fillet'  0.09 ms
'find_front_face_torodial_edges_to_fillet'  0.10 ms
'filleted_envelope'  9.80 ms
'find_envelope_back_face'  1487.04 ms
'find_envelope_front_face'  10.03 ms
'find_front_face_midpoint'  0.25 ms
'envelope_front_face_id'  10.14 ms
'find_envelope_back_face'  1576.92 ms
'find_envelope_front_face'  9.69 ms
'find_front_face_midpoint'  0.25 ms
'envelope_front_face_id'  10

'exstrude_and_cut_solids'  607.09 ms
'chop_up_poloidally'  1443.01 ms
'make_cylinder_slice'  0.19 ms
'find_envelope_back_face'  2491.42 ms
'find_envelope_back_face'  1681.14 ms
'find_envelope_front_face'  16.96 ms
'envelope_front_face_id'  17.17 ms
'find_end_cap_faces'  0.14 ms
'make_cylinder_slice'  1.28 ms
'make_cylinder_slice'  0.20 ms
'make_cylinder_slice'  0.25 ms
'make_cylinder_slice'  0.31 ms
'make_cylinder_slice'  0.23 ms
'chop_off_first_wall_armour'  296.21 ms
'chop_off_back_plates'  461.75 ms
'make_cylinder_slice'  2.13 ms
'convert_distances_into_cumlative_distances'  0.03 ms
'find_envelope_back_face'  2629.55 ms
'find_envelope_front_face'  12.20 ms
'find_front_face_midpoint'  0.33 ms
'make_cylinder_slice'  0.92 ms
'make_cylinder_slice'  0.31 ms
'make_cylinder_slice'  0.30 ms
'save_components_as_step'  585.39 ms
'chop_of_end_caps'  186.61 ms
'find_envelope_back_face'  3235.05 ms
'find_envelope_front_face'  15.55 ms
'find_front_face_midpoint'  0.31 ms
'chop_of_end_caps'  194.9

'read_in_envelope_file'  10.28 ms
'save_components_as_stl'  989.04 ms
'find_envelope_back_face'  1976.31 ms
'find_envelope_front_face'  11.56 ms
'find_end_cap_faces'  0.10 ms
'exstrude_and_cut_solids'  2417.70 ms
'chop_up_poloidally'  5186.14 ms
'make_cylinder_slice'  0.13 ms
'find_envelope_back_face'  2000.71 ms
'find_envelope_front_face'  11.60 ms
'find_front_face_midpoint'  0.18 ms
'find_envelope_back_face'  1983.33 ms
'find_envelope_front_face'  11.66 ms
'find_end_cap_faces'  0.12 ms
'find_envelope_back_face'  1495.77 ms
'find_envelope_front_face'  11.73 ms
'find_front_face_midpoint'  0.22 ms
'read_in_envelope_file'  11.41 ms
'envelope_front_face_id'  15.38 ms
'chop_of_end_caps'  206.65 ms
'find_envelope_back_face'  1493.99 ms
'find_envelope_front_face'  11.69 ms
'find_front_face_midpoint'  0.20 ms
'envelope_front_face_id'  11.83 ms
'make_cylinder_slice'  1.98 ms
'make_cylinder_slice'  0.18 ms
'make_cylinder_slice'  0.33 ms
'make_cylinder_slice'  0.42 ms
'make_cylinder_slice'  0.35

'find_front_face_polodial_edges_to_fillet'  0.09 ms
'envelope_front_face_id'  14.05 ms
'find_front_face_torodial_edges_to_fillet'  0.09 ms
'find_envelope_back_face'  1643.32 ms
'filleted_envelope'  16.18 ms
'make_cylinder_slice'  0.47 ms
'make_cylinder_slice'  0.32 ms
'find_envelope_back_face_id'  448.25 ms
'find_front_face_polodial_edges_to_fillet'  0.10 ms
'find_front_face_torodial_edges_to_fillet'  0.09 ms
'filleted_envelope'  15.43 ms
'find_envelope_back_face'  1316.18 ms
'find_envelope_front_face'  13.79 ms
'find_end_cap_faces'  0.13 ms
'find_envelope_back_face'  639.18 ms
'find_envelope_front_face'  20.94 ms
'envelope_front_face_id'  20.96 ms
'find_end_cap_faces'  0.12 ms
'chop_off_back_plates'  856.80 ms
'convert_distances_into_cumlative_distances'  0.04 ms
'chop_off_first_wall_armour'  291.30 ms
'find_envelope_back_face'  597.26 ms
'find_envelope_front_face'  20.57 ms
'envelope_front_face_id'  21.14 ms
'find_end_cap_faces'  0.13 ms
'make_cylinder_slice'  1.85 ms
'chop_off_first

'find_envelope_back_face'  481.41 ms
'find_envelope_front_face'  11.25 ms
'find_front_face_midpoint'  0.24 ms
'envelope_front_face_id'  11.41 ms
'exstrude_and_cut_solids'  2219.63 ms
'find_envelope_back_face'  972.95 ms
'find_envelope_front_face'  18.26 ms
'envelope_front_face_id'  17.06 ms
'find_end_cap_faces'  0.13 ms
'find_envelope_back_face'  967.87 ms
'find_envelope_front_face'  15.77 ms
'envelope_front_face_id'  18.13 ms
'find_end_cap_faces'  0.15 ms
'find_envelope_back_face'  1179.46 ms
'find_envelope_front_face'  9.88 ms
'find_end_cap_faces'  0.12 ms
'find_envelope_back_face'  879.56 ms
'chop_off_first_wall_armour'  282.69 ms
'find_envelope_back_face_id'  568.60 ms
'find_front_face_polodial_edges_to_fillet'  0.08 ms
'find_front_face_torodial_edges_to_fillet'  0.07 ms
'filleted_envelope'  12.95 ms
'chop_off_first_wall_armour'  260.71 ms
'chop_off_back_plates'  390.71 ms
'convert_distances_into_cumlative_distances'  0.03 ms
'find_envelope_back_face'  652.86 ms
'find_envelope_fron

('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
'add_cooling_pipes_to_div'  3612.92 ms
'make_cylinder_slice'  0.29 ms
'make_cylinder_slice'  0.16 ms
'make_cylinder_slice'  0.13 ms
'make_cylinder_slice'  0.18 ms
'make_cylinder_slice'  0.39 ms
'make_cylinder_slice'  0.66 ms
'make_cylinder_slice'  7.40 ms
'make_cylinder_slice'  0.18 ms
'make_cylinder_slice'  0.19 ms
'make_cylinder_slice'  0.19 ms
'make_cylinder_slice'  0.25

In [10]:
extra_parts = read_in_step_files_and_save_as_seperate_stl_files(read_folder='/home/jshim/Eurofusion_baseline_2016/reactor_step_files',
                                                                write_folder=os.path.join(output_directory,'stl'),
                                                                ignore_files=['shell.stp'])


('error making folder', '/home/jshim/detailed_HCLL/stl')
('read_folder', '/home/jshim/Eurofusion_baseline_2016/reactor_step_files')
['shield.stp', 'vacuum_1st_layer.stp', 'divertor_4th_layer.stp', 'divertor_3rd_layer.stp', 'divertor_2nd_layer.stp', 'vaccum_vessel_shield.stp', 'plasma.stp', 'divertor_1st_layer.stp', 'blanket_support.stp', 'central_solenoid.stp', 'vacuum_2nd_layer.stp', 'manifolder.stp', 'ports.stp', 'shell.stp', 'vacuum_3rd_layer.stp', 'tf_coils.stp', 'tf_case.stp']
shield.stp
vacuum_1st_layer.stp
divertor_4th_layer.stp
divertor_3rd_layer.stp
divertor_2nd_layer.stp
vaccum_vessel_shield.stp
plasma.stp
divertor_1st_layer.stp
blanket_support.stp
central_solenoid.stp
vacuum_2nd_layer.stp
manifolder.stp
ports.stp
vacuum_3rd_layer.stp
tf_coils.stp
tf_case.stp


In [11]:
list_of_detailed_modules_parts.update(extra_parts)


AttributeError: 'list' object has no attribute 'update'